<a href="https://colab.research.google.com/github/rushivarun/reinforcement_learning/blob/master/CartPole_v0_trainingNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install pyglet==1.3.2

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (41.0.1)


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [8]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

W0808 12:35:21.044443 140542809208704 abstractdisplay.py:151] xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [5]:
import random
import tflearn 
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median 
from collections import Counter

LR = 1e-3
env = wrap_env(gym.make("CartPole-v0"))
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000

def some_random_games():
  for episode in range(5):
    env.reset()
    for t in range(goal_steps):
      env.render()
      action = env.action_space.sample()
      observation, reward, done, info = env.step(action)
      if done:
        break
        
        
        
        
def initial_population():
  training_data = []
  scores = []
  accepted_scores = []
  
  for _ in range(initial_games):
    score = 0
    game_memory = []
    prev_observation = []
    
    for _ in range(goal_steps):
      action = random.randrange(0, 2)
      observation, reward, done, info = env.step(action)
      if len(prev_observation) > 0:
        game_memory.append([prev_observation, action])
      prev_observation = observation
      score += reward
      if done:
        break
    if score >= score_requirement:
      accepted_scores.append(score)
      
      for data in game_memory:
        if data[1] == 0:
          output = [1, 0]
        elif data[1] == 1:
          output = [0, 1]
        training_data.append([data[0], output])
    env.reset()
    scores.append(score)
  
  training_data = np.array(training_data)
  np.save('train.npy', training_data)
  
  
  print('mean of all scores are', mean(accepted_scores))
  print('median of all scores are', median(accepted_scores))
  print(Counter(accepted_scores))
  print(training_data)
  
  return training_data


# initial_population()
        
#     env.close()
#     show_video()

# some_random_games()

# env.close()
# show_video()      

W0808 07:12:03.485691 139718035486592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0808 07:12:03.487262 139718035486592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0808 07:12:03.496760 139718035486592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0808 07:12:03.503586 139718035486592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0808 07:12:03.513332 139718035486592 deprecation_wrapper.py:119] From /usr/local/lib/

In [7]:
import random
import tflearn 
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median 
from collections import Counter

LR = 1e-3
env = wrap_env(gym.make("CartPole-v0"))
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000


def training_population():
  training_data = []
  scores = []
  accepted_scores = []
  
  for _ in range(initial_games):
    score = 0
    game_memory = []
    prev_observation = []
    
    for _ in range(goal_steps):
      action = random.randrange(0, 2)
      observation, reward, done, info = env.step(action)
      if len(prev_observation) > 0:
        game_memory.append([prev_observation, action])
      prev_observation = observation
      score += reward
      
      if done:
        break 
    
    if score >= score_requirement:
      accepted_scores.append(score)
    
    
    
    for data in game_memory:
      if data[1] == 1:
        output = [0, 1]
      elif data[1] == 0:
        output = [1, 0]
      
      training_data.append([data[0], output])
      
    env.reset()
    scores.append(score)
      
  training_data_array = np.array(training_data)
  np.save('training_array.npy', training_data_array)
    
    
  print("printing metrics of session........")
  print('mean of accepted scores are: ', mean(accepted_scores))
  print('median of accepted scores are: ', median(accepted_scores))
  print(Counter(accepted_scores))
  
  print("training_data")
  print(training_data_array)
  
  
  return training_data_array


training_population()
    

printing metrics of session........
mean of accepted scores are:  61.80266666666667
median of accepted scores are:  58.0
Counter({50.0: 34, 53.0: 26, 54.0: 24, 52.0: 22, 58.0: 20, 59.0: 20, 55.0: 19, 51.0: 18, 56.0: 17, 57.0: 17, 61.0: 13, 64.0: 12, 60.0: 12, 63.0: 12, 62.0: 11, 69.0: 9, 75.0: 8, 68.0: 7, 65.0: 5, 85.0: 5, 66.0: 5, 73.0: 4, 77.0: 4, 76.0: 4, 72.0: 3, 74.0: 3, 79.0: 3, 67.0: 3, 80.0: 3, 70.0: 3, 83.0: 3, 88.0: 3, 103.0: 2, 81.0: 2, 95.0: 2, 86.0: 2, 91.0: 2, 97.0: 1, 71.0: 1, 110.0: 1, 92.0: 1, 87.0: 1, 126.0: 1, 78.0: 1, 113.0: 1, 89.0: 1, 82.0: 1, 134.0: 1, 108.0: 1, 96.0: 1})
training_data
[[array([ 0.0319193 , -0.21401748,  0.04890724,  0.31611212])
  list([0, 1])]
 [array([ 0.02763895, -0.01962505,  0.05522948,  0.03924511])
  list([0, 1])]
 [array([ 0.02724645,  0.17466321,  0.05601438, -0.23551369])
  list([1, 0])]
 ...
 [array([ 0.17496474, -0.15227265, -0.18110355, -0.18227296])
  list([0, 1])]
 [array([ 0.17191928,  0.04491626, -0.18474901, -0.5261728 ])
  lis

[[array([ 0.0319193 , -0.21401748,  0.04890724,  0.31611212]), [0, 1]],
 [array([ 0.02763895, -0.01962505,  0.05522948,  0.03924511]), [0, 1]],
 [array([ 0.02724645,  0.17466321,  0.05601438, -0.23551369]), [1, 0]],
 [array([ 0.03073971, -0.02121244,  0.05130411,  0.07429859]), [0, 1]],
 [array([ 0.03031546,  0.1731379 ,  0.05279008, -0.20176656]), [1, 0]],
 [array([ 0.03377822, -0.02269774,  0.04875475,  0.10709076]), [0, 1]],
 [array([ 0.03332426,  0.17169287,  0.05089656, -0.16982045]), [0, 1]],
 [array([ 0.03675812,  0.36605077,  0.04750015, -0.44602265]), [0, 1]],
 [array([ 0.04407914,  0.56046967,  0.0385797 , -0.72336227]), [1, 0]],
 [array([ 0.05528853,  0.36483596,  0.02411246, -0.41879044]), [1, 0]],
 [array([ 0.06258525,  0.16938077,  0.01573665, -0.11860452]), [0, 1]],
 [array([ 0.06597287,  0.36427376,  0.01336456, -0.40628142]), [0, 1]],
 [array([ 0.07325834,  0.55920367,  0.00523893, -0.69472111]), [0, 1]],
 [array([ 0.08444241,  0.75425257, -0.00865549, -0.9857502 ]), [

In [10]:
import random
import tflearn 
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median 
from collections import Counter



LR = 1e-3
env = wrap_env(gym.make("CartPole-v0"))
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000




def initial_population_2():
    # [OBS, MOVES]
    training_data = []
    # all scores:
    scores = []
    # just the scores that met our threshold:
    accepted_scores = []
    # iterate through however many games we want:
    for _ in range(initial_games):
        score = 0
        # moves specifically from this environment:
        game_memory = []
        # previous observation that we saw
        prev_observation = []
        # for each frame in 200
        for _ in range(goal_steps):
            # choose random action (0 or 1)
            action = random.randrange(0,2)
            # do it!
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done: break

        # IF our score is higher than our threshold, we'd like to save
        # every move we made
        # NOTE the reinforcement methodology here. 
        # all we're doing is reinforcing the score, we're not trying 
        # to influence the machine in any way as to HOW that score is 
        # reached.
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                # convert to one-hot (this is the output layer for our neural network)
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                    
                # saving our training data
                training_data.append([data[0], output])

        # reset env to play again
        env.reset()
        # save overall scores
        scores.append(score)
    
    # just in case you wanted to reference later
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    # some stats here, to further illustrate the neural network magic!
    print('Average accepted score:',mean(accepted_scores))
    print('Median score for accepted scores:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data









def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')

    return model
  
  

  
def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    
#       print('printing x', X)
#       print('length of x: ', len(X))
#   print('length of x complicated: ', len(X[0]))
#   print('training data is compl: ', training_data[0][0])
  
    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input': X}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_learning')
    return model

training_data = initial_population_2()
model = train_model(training_data)



scores = []
choices = []

for each_game in range(10):
  score = 0
  game_memory = []
  prev_obs = []
  env.reset()
  
  for _ in range(goal_steps):
    env.render()
    
    if len(prev_obs) == 0:
      action = random.randrange(0, 2)
      
    else:
      print(model.predict(prev_obs.reshape(-1, len(prev_obs), 1)))
      action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
    
    choices.append(action)
    
    new_observation, reward, done, info = env.step(action)
    
    prev_obs = new_observation
    game_memory.append([new_observation, action])
    score += reward
    
    if done:
      break
    
    scores.append(score)
    
    print('avg score lol.', mean(scores))
    
    
show_video()


    
      



Average accepted score: 62.55725190839695
Median score for accepted scores: 58.0
Counter({52.0: 30, 53.0: 28, 51.0: 26, 50.0: 25, 54.0: 23, 58.0: 18, 56.0: 18, 55.0: 18, 57.0: 17, 61.0: 15, 62.0: 14, 65.0: 14, 69.0: 11, 60.0: 11, 59.0: 11, 63.0: 9, 64.0: 8, 66.0: 7, 72.0: 7, 71.0: 6, 82.0: 5, 68.0: 5, 74.0: 5, 75.0: 5, 76.0: 4, 73.0: 4, 88.0: 4, 80.0: 4, 70.0: 3, 78.0: 3, 83.0: 3, 79.0: 2, 91.0: 2, 98.0: 2, 84.0: 2, 67.0: 2, 92.0: 1, 77.0: 1, 89.0: 1, 111.0: 1, 81.0: 1, 105.0: 1, 85.0: 1, 104.0: 1, 90.0: 1, 113.0: 1, 102.0: 1, 114.0: 1, 94.0: 1, 108.0: 1, 87.0: 1, 93.0: 1, 120.0: 1, 101.0: 1, 106.0: 1, 119.0: 1, 95.0: 1, 100.0: 1})
---------------------------------
Run id: openai_learning
Log directory: log/
---------------------------------
Training samples: 72576
Validation samples: 0
--


InvalidArgumentError: ignored